In [60]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    make_scorer, classification_report
)
import numpy as np

X,y =load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

pipeline = Pipeline([
    ('scaler',StandardScaler()), 
    ('classifier', RandomForestClassifier())
])

dt = DecisionTreeClassifier(max_depth=1)

param_grid = [
    {'classifier': [LogisticRegression()],
    'classifier__C' : [0.1,1,10]},
    
    {'classifier': [RandomForestClassifier()],
    'classifier__n_estimators':[50,100,200], 
    'classifier__max_depth': [None, 5, 10]},

    {'classifier':[SVC()], 
    'classifier__kernel': ['rbf', 'linear'],
    'classifier__C':[0.1, 1, 10]}, 

    {'classifier': [AdaBoostClassifier()],
    'classifier__estimator': [dt]}
]
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision':make_scorer(precision_score, average='macro'),
    'recall': make_scorer(recall_score, average='macro'),
    'f1': make_scorer(f1_score, average='macro')
}
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring=scoring,refit='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)



print(f'Best Model: {grid_search.best_params_}')
print(f'Best Score: {grid_search.best_score_}')

best_model = grid_search.best_estimator_
test_accuracry = best_model.score(X_test, y_test)
print(f'Test Accuracy: {test_accuracry:.2f}')

cv_results = grid_search.cv_results_
metrics = ['accuracy', 'precision', 'recall', 'f1']

best_scores = {metric: np.max(cv_results[f'mean_test_{metric}']) for metric in metrics}

# Display Best Scores for Each Metric
print("\n🔹 Best Scores Across Metrics:")
for metric, score in best_scores.items():
    print(f"{metric.capitalize()}: {score:.4f}")

Best Model: {'classifier': SVC(), 'classifier__C': 0.1, 'classifier__kernel': 'linear'}
Best Score: 0.975
Test Accuracy: 0.93

🔹 Best Scores Across Metrics:
Accuracy: 0.9750
Precision: 0.9793
Recall: 0.9750
F1: 0.9749
